함수호출

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import class_informs as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm
#import seaborn as sbn

import time,datetime
import cmath


import re
import matplotlib.pyplot as plt

데이터 전처리

In [2]:
#timeseries data
all_data=pd.read_csv('data/Timeseries_data_SP500.csv')

all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

date_list=list(set(list(all_data['date'])))
date_list.sort()


dic_data = {k: v for k, v in all_data.groupby('date')}


results_=pd.read_csv('data/results_template.csv')
results_=results_.rename(columns={'DATE':'date','SEDOL':'sedol','WEIGHTS':'w','FOUR_WEEKLY_RETURN':'r'})
results_['date']=pd.to_datetime(results_['date'])
dic_results_={k: v for k, v in results_.groupby('date')}


def preprocessing(date):
    
    using_dic = dic_data[pd.to_datetime(date)]
    using_dic['index']=list(using_dic.index)

    asset_list = []
    for i in using_dic["sedol"]:
        asset_list.append(i)


    dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

    dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

    dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

    dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }

    dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

    for i in using_dic.index:
        dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
        dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])



    #risk:cov_mat
    date_str=str(date)[:10]
    risk_data=pd.read_csv('data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    #         return risk_data,risk_sedol,risk_mat

    
    alpha = []


    for i in risk_sedol:

        alpha.append(-1*dic_sedol_as[i])

    dic_r={dic_results_[date]["sedol"][i] : dic_results_[date]["r"][i] for i in using_dic.index}

    return using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha

print("complete")

complete


함수 정의

In [3]:
def cal_obj1(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj

def cal_obj(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    sol_d = list(np.array(sol) - np.array(using_dic['bw']))
    s_rr = np.dot(sol_d, risk_mat)
    s_r = np.dot(s_rr, sol_d)
    s_a = np.dot(sol_d, as_*10000)
    obj = s_r-s_a
    return obj



def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []

    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)
    return candi



def update(sol,lim_time,as_,w_pre,O_scale) :

    kkset=time.time()
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    start_time = time.time()
    
    while(True):

        if time.time() - start_time > lim_time:
            break

        ex.set_con(cons=con)
        print('kkset time',time.time()-kkset)
      
        new_list2 = []
        new_list3 = []

        try:
            kkcplex=time.time()
            aabbaa = ex.solve(0.005)

            print('clexsolve time : ',time.time()-kkcplex)
            cplex_t=time.time()-kkcplex
            
            if aabbaa ==1234:
                break

            if aabbaa != 0:
                aaaa = aabbaa[0]
                bbbb= aabbaa[1]
                for k in risk_sedol :
                    new_list2.append(aaaa[k])
                for k in risk_sedol :
                    new_list3.append(bbbb[k])
                new_list = new_list2

                result_list.append(aabbaa[2])
                result_list2.append(aabbaa[0])
                print("obj_val")
                print(cal_obj1(new_list3, risk_mat, as_))
                con = select_candi(new_list)
                print("cplex_val")
                print(aabbaa[2])

            else:
                print("TE infeasible")
                con = select_candi(new_list)


        except:
            print("cplex no solution")
            con = select_candi(new_list)
            cplex_t=0


    return result_list,result_list2,cplex_t

def update2(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []

    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []

        aabbaa = ex.solve(0.01)


        if aabbaa != 0:
            aaaa = aabbaa[0]
            bbbb= aabbaa[1]
            for k in risk_sedol :
                new_list2.append(aaaa[k])
            for k in risk_sedol :
                new_list3.append(bbbb[k])
            new_list = new_list2

            result_list.append(aabbaa[2])
            result_list2.append(aabbaa[0])
            con = select_candi(new_list)


        else:
            print("TE infeasible")
            con = select_candi(new_list)





    return result_list,result_list2


# def sample_Z(m, n):
#     return Variable(torch.Tensor(np.random.rand(m, n)))

        
# def xavier_init(size):
#     in_dim = size[0]
#     xavier_stddev = 1. / np.sqrt(in_dim / 2.)
#     return np.random.normal(size=size, scale=xavier_stddev)


# class Generator(nn.Module):
#     def __init__(self,  in_dim=10, hd_dim=50, out_dim=500):
#         super(Generator, self).__init__()
        
#         self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
#         self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
#         #print('w1',self.G_W1)

        
#         self.G_W12 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
#         self.G_b12 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
#         self.G_W13 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
#         self.G_b13 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
#         self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
#         self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

#     def forward(self, x=None):

#         x = F.relu( x @ self.G_W1 + self.G_b1)
#         x = F.relu( x @ self.G_W12 + self.G_b12)
#         #x = F.relu( x @ self.G_W13 + self.G_b13)
#         x = x @ self.G_W2 + self.G_b2


#         return F.sigmoid(x*0.1)


# class Discriminator(nn.Module):
#     def __init__(self, in_dim=500,train=False):
#         super(Discriminator, self).__init__()
#         self.tot_infs = 0.0
#         self.tot_infs_nor = 0.0
#         self.train = train
#         self.x_threshold = 0.001
#         self.inf_scale = 100.0
#         self.Zstar = 1000
        
#         self.in_dim = in_dim

#         np.random.seed(0)
#         self.Omega = Variable(torch.from_numpy(risk_mat)).float()

#         self.card_upper = 70.
#         self.card_lower = 69.
        
        
#     def forward(self, x):
#         tot_infs = Variable(torch.zeros(x.size()[0]))
#         x_bw = Variable(torch.from_numpy(bw).float())
#         alp = Variable(torch.Tensor(alpha).float())   
#         bet = Variable(torch.Tensor(beta).float())
 
#         # (4)
#         tot_infs += F.relu(1. - torch.sum(x,1))
#         tot_infs += F.relu(torch.sum(x,1) - 1.)
        
#         # (5)
#         tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)

#         # (6)
#         for j in range(nbsector):
#             k= (x-x_bw) @ Variable(
#                 torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
#             l = torch.abs(k)
#             tot_infs += F.relu(l - 0.1)
            
#         # (7)
#         for j in range(nbmq):
#             k = (x-x_bw) @ Variable(
#                 torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
#             l = torch.abs(k)
#             tot_infs += F.relu(l - 0.1)
            
#         # (8)
#         k = (x - x_bw)@bet
#         l = torch.abs(k)
#         tot_infs += F.relu(l - 0.1)
        
#         # (9) lb <= card(x) <= b
#         if self.train:
#             num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
#             tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
#         else:
#             num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
#             tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
#         # (10)
#         l = 0.5*torch.sum(torch.abs(x - x_bw))
#         tot_infs += F.relu(0.6 - l)
#         #tot_infs += F.relu(l - 1.)
    
        
#         # (11) a<=dQd <= b
#         dQ = (x-x_bw) @ self.Omega
#         size_x = x.size()
#         dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
#         tot_infs += F.relu(dQd - 0.01)*0.3
#         tot_infs += F.relu(0.0025 - dQd)*0.3
        
        
#         # min dQd + ad (<= Zstar)
#         l = (x-x_bw)@alp

#         MONTH =0
#         turnover=x_bw
        
#         if MONTH > 0 :
#             tot_infs += (F.relu(100.*dQd - 100.*l -0.5*turnover - self.Zstar))
#             self.objs = 100.*dQd - 100.*l -0.5*turnover
#         else:
#             tot_infs += (F.relu(100.*dQd - 100.*l  - self.Zstar ))
#             self.objs = 100.*dQd - 100.*l
        
        
#         fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))  

        
#         return fea_probs


def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


In [4]:
def w_i_pre_sum(w_dic):
   
    w_pre = 0  
    
    for i in w_dic.keys():
        w_pre += w_dic[i]*(dic_r[i]+1)
        
        
    w_i_pre_dic = {}
    for i in w_dic.keys():
        w_i_pre_dic.update({i:(float(w_dic[i])*(dic_r[i]+1))/w_pre})

    return w_i_pre_dic



def turnover(w_dic,w_dic2, date, risk_sedol, dic_sedol_as):
    
    mon = date_list[date]
    mon2 = date_list[date-1]

    rotp_t0 = 0
    for i in risk_sedol:
        rotp_t0 += w_dic[i]*dic_r_all[mon2][i]
        
    w_pre = 0
    for i in risk_sedol:
        w_pre += w_dic[i]*(dic_r_all[mon2][i]+1)

    w_i_pre = {}
    w_i_pre_sum = 0
    for i in risk_sedol:
        w_i_pre.update({i:(w_dic[i]*(dic_r_all[mon2][i]+1))/w_pre})
        w_i_pre_sum += (w_dic[i]*(dic_r_all[mon2][i]+1))/w_pre
    
    pre_list = set(list(w_i_pre.keys()))
    pre_list2 = set(list(w_dic2.keys()))

    turnover = 0

    add_key_1 = pre_list-pre_list2
    add_key_2 = pre_list2-pre_list

    for i in add_key_2:
        w_i_pre.update({i:0})

    for i in add_key_1:
        w_dic2.update({i:0})

    pre_list.union(pre_list2)
    for i in pre_list:
        turnover += abs(w_dic2[i]-w_i_pre[i])

    rOTP = 0

    for i in dic_sedol_as.keys():
        rOTP += w_dic2[i]*dic_r_all[mon][i]

    r_Txadj_t = rOTP - turnover*0.005
    
    print("turnover : " + str(turnover))
    print("r_OTP : " + str(rOTP))
    print("r_Txadj_t : " + str(r_Txadj_t))
    
#     result=[]
#     result.append(turnover,rOTP,r_Txadj_t)
#     result.append(w_i_pre_sum)
    
#     return result

    return r_Txadj_t



def IR_TE(result,bench_t):
    cal1 = 1
    cal2 = 1
    list_std=np.array([])

    for i in range(len(result)):
        cal1 = cal1*(1+result[i])
        cal2 = cal2*(1+bench_t[i])
        list_std = np.append(list_std,result[i]-bench_t[i])
    
    IR = (cal1-cal2)/np.std(list_std)
    
    TE = math.sqrt(13)*np.std(list_std)
    
#     result = np.array([])
#     result = np.append(IR,TE)
#     return result

    return IR



def R_SR(result,bench,n):
    R_CUM = 1
    B_CUM = 1
    for i in range(len(result)):
        R_CUM = R_CUM*(1+result[t])
        
    for i in range(len(bench)):
        B_CUM = B_CUM*(1+bench[t])
    
    R_CUM = R_CUM-1
    
    B_CUM = B_CUM-1
    
    R_ANN = (1+R_CUM)**(13/n) - 1
    
    B_ANN = (1+B_CUM)**(13/n) - 1
    
    R_EXAN = R_ANN-B_ANN
    
    r_t = np.array(result)
    
    SR = R_CUM/np.std(r_t)

solve!

In [7]:
import generator
import discriminator

#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []
n = 1
fini = True
k = 0
w_pre = {}
opt_return_list = []
sol_all=[]

res_tem = pd.read_csv('data/results_template.csv')
w_pre = {}
for i in range(492) :
    w_pre.update({res_tem['SEDOL'][i]:res_tem['WEIGHTS'][i]})

############################################
while(fini):
    

    
    print(n, "period")

    date=date_list[n]
    init_set='GAN' #initial set by GAN, cplex, random

    c_time = 10. #cplex time for initial set
    gan_rot = 1800 #GAN epoch size for initial set
############################################


#preprocessing
    using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha=preprocessing(date)

#     if n == 0:
#         res_tem = pd.read_csv('data/results_template.csv')
#         w_pre = {}
#         for i in range(492) :
#             w_pre.update({res_tem['SEDOL'][i]:res_tem['WEIGHTS'][i]})

    if init_set=='GAN':

        bw=np.zeros(len(dic_beta))
        beta=np.zeros(len(dic_beta))
        alpha=np.zeros(len(dic_beta))
        
        for i in range(len(risk_sedol)):
            bw[i]=dic_bench[risk_sedol[i]]
            beta[i]=dic_beta[risk_sedol[i]]
            alpha[i]=dic_sedol_as[risk_sedol[i]]

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)
        X_dim = nbasset
        HL_dim = 50
        Z_dim = 10
        w_pre_gan=np.zeros(len(asset_list))
        for i,j in enumerate(asset_list):
            if j in w_pre.keys(): w_pre_gan[i]=w_pre[j]
            else:w_pre_gan[i]=0.

        car=50
#         xavier2=xavier
       
        if n == 1:
            np.random.seed()
            G = generator.Generator(Z_dim, HL_dim, X_dim,generator.xavier_init([Z_dim,HL_dim]))
        else :
    
            np.random.seed()
            G = generator.Generator(Z_dim, HL_dim, X_dim,xavier2)
            

        sample_D = discriminator.Discriminator(in_dim=X_dim, train=False, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                               ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
        train_D = discriminator.Discriminator(in_dim=X_dim, train=True, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                              ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
        G_solver = optim.Adam(G.parameters(), lr=2e-3)

        mb_size = 128

        best_obj = 100000.0
        best_sols = []

        losses = []

        start_time = time.time()

        feasible_threshold = 0.95

        adaptive_inf_scale = float(nbasset)
        rotation=1800

        for it in range(rotation):
            if it % 900 == 0:
                sample_noise = generator.sample_Z(128, Z_dim)
                samples = G(sample_noise)

                sample_D.Zstar = best_obj
                sample_D.inf_scale = adaptive_inf_scale

                sample_D_values = sample_D(samples).data.numpy()
                sample_obj_values = sample_D.objs.data.numpy()

                fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
                sols = samples.data.numpy()[sample_D_values > feasible_threshold]

                if len(fea_sample_obj_values > 0):
                    best_idx = np.argmin(fea_sample_obj_values)
                    if np.min(fea_sample_obj_values) < best_obj:
                        best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                        best_sols.append(sols[best_idx])

            z = Variable(torch.randn(mb_size, Z_dim))

            train_D.Zstar = best_obj
            train_D.inf_scale = adaptive_inf_scale

            G_loss = -torch.mean(torch.log(train_D(G(z))))

            G.zero_grad()
            G_loss.backward()
            G_solver.step()
            
        
        xavier2=G.xavier
   
        best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


# solve & update
    sol=sol
    as_ = using_dic['as']   
    
    results = update(sol,30.,as_,w_pre,1)

    result = results[0]
    
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)
        sol_all.append(result2)

        w_pre = w_i_pre_sum(result2)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화------")
        print(result)
        n += 1

    except:
    
        infeasi += 1
        
        
    if n == 11:
        fini = False
    k += 1
print("feasible 횟수 : ",feasi)
print("저장된 objective value")
print(list_feasi_obj)
print("infeasible 횟수 : ",infeasi)
print(opt_return_list)

1 period
kkset time 12.656800031661987
O : 1.8240221935884091
23.084067913228353
AC
0.884016741619
TE
[ 21.26085147]
0.728132414142  wupsum
1.00000006017  wsum!
17  wupdic_len
O : 1.8485876784790012
26.395090827229204
TE :  [ 24.5500431]
AS :  0.886064145968
O : 116754.04763723482
116784.70453022029
TE :  [ 30.66412514]
AS :  0.884151659579
Bs infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  1.3734500408172607
cplex no solution
kkset time 18.09268093109131
O : 1.7934918238874917
22.630944139992394
AC
0.885174371839
TE
[ 20.8380641]
0.720217185347  wupsum
1.00000005418  wsum!
17  wupdic_len
O : 1.825951372731775
26.0161890853122
TE :  [ 24.1943037]
AS :  0.885579556522
O : 100528.6789864204
100559.11307429633
TE :  [ 30.44145462]
AS :  0.884928307
Bs infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  1.3099431991577148
cplex no solution
kkset time 22.463134050369263
O : 1.7633217443118885
22.263166799397684
AC
0.8846759

kkset time 12.918951988220215
O : 1.8512834624760617
27.183263391825665
AC
0.887584908
TE
[ 25.36103547]
clexsolve time :  0.41453981399536133
obj_val
25.3570025472
cplex_val
27.183263391825665
kkset time 16.223067045211792
O : 1.8447718620318
27.782943197161377
AC
0.914527623
TE
[ 25.9817741]
clexsolve time :  0.4245419502258301
obj_val
25.9680002124
cplex_val
27.782943197161377
kkset time 18.83261203765869
O : 1.8406890572333865
26.8915760696553
AC
0.914752894
TE
[ 25.09564938]
clexsolve time :  0.38930511474609375
obj_val
25.0869781154
cplex_val
26.8915760696553
kkset time 21.332318782806396
cplex no solution
kkset time 24.31581711769104
cplex no solution
kkset time 26.63991117477417
cplex no solution
kkset time 27.42403292655945
cplex no solution
kkset time 30.29505705833435
cplex no solution
kkset time 31.01626420021057
cplex no solution
kkset time 33.547078132629395
cplex no solution
kkset time 34.32271981239319
cplex no solution
kkset time 37.24470901489258
cplex no solution
kks

kkset time 12.553219079971313
O : 1.9570242450965645
39.89039013365149
AC
0.901470931142
TE
[ 37.93649339]
clexsolve time :  0.4127800464630127
obj_val
37.9595925495
cplex_val
39.89039013365149
kkset time 13.603165864944458
O : 1.8528306556131822
30.802712037687254
AC
0.906471560388
TE
[ 28.95443758]
clexsolve time :  0.3964059352874756
obj_val
28.9663815389
cplex_val
30.802712037687254
kkset time 15.71268606185913
cplex no solution
kkset time 18.107074975967407
O : 1.8524493156301018
24.498015054129716
AC
0.901079796056
TE
[ 22.63718751]
0.788958751812  wupsum
1.00000011376  wsum!
18  wupdic_len
O : 1.8704205445376427
25.85927713469832
TE :  [ 23.97624814]
AS :  0.901644021975
O : 1.8878315904898728
27.83540390485994
TE :  [ 25.9322164]
AS :  0.902168178618
O : 1.8817044128456768
26.67511943571102
TE :  [ 24.77954354]
AS :  0.901759734815
O : 1.8861963158202852
27.198850753535584
TE :  [ 25.29794473]
AS :  0.902168026503
O : 1.884640709400429
26.925131632892136
TE :  [ 25.02618828]
AS

kkset time 12.979424953460693
cplex no solution
kkset time 19.205713748931885
cplex no solution
kkset time 24.600193977355957
cplex no solution
kkset time 30.31124496459961
cplex no solution
kkset time 36.5481538772583
cplex no solution
kkset time 42.14490270614624
cplex no solution
4 period
kkset time 12.034870862960815
O : 1.9824215430258552
53.12261730072355
AC
0.906687563841
TE
[ 51.17234922]
clexsolve time :  0.35854387283325195
obj_val
51.1534478532
cplex_val
53.12261730072355
kkset time 12.876145839691162
O : 59800.46445191609
59877.84511586815
AC
0.922517894
TE
[ 77.41547802]
infeasible
st_6 infeasible and infeasible sector is :  Health Care
clexsolve time :  0.45032691955566406
cplex no solution
kkset time 14.201337099075317
O : 58602.33220887646
58665.451955397744
AC
0.921478752
TE
[ 63.13856337]
infeasible
st_6 infeasible and infeasible sector is :  Health Care
clexsolve time :  0.42319798469543457
cplex no solution
kkset time 16.260343074798584
O : 111016.15841403203
111066

kkset time 13.776166915893555
O : 108457.79170601851
108507.44938780427
AC
0.89600288218
TE
[ 49.65070557]
infeasible
st_6 infeasible and infeasible sector is :  Industrials
clexsolve time :  0.4511089324951172
cplex no solution
kkset time 17.517958879470825
O : 106839.60534902805
106889.36356042264
AC
0.896442437071
TE
[ 49.75135603]
infeasible
st_6 infeasible and infeasible sector is :  Industrials
clexsolve time :  0.4331831932067871
cplex no solution
kkset time 21.74061894416809
O : 102890.4541885868
102940.10924227817
AC
0.896513550676
TE
[ 49.64703924]
infeasible
st_6 infeasible and infeasible sector is :  Industrials
clexsolve time :  0.42545270919799805
cplex no solution
kkset time 24.57548189163208
O : 105592.90832736339
105642.75496765667
AC
0.89531746581
TE
[ 49.83498579]
infeasible
st_6 infeasible and infeasible sector is :  Industrials
clexsolve time :  0.4301321506500244
cplex no solution
kkset time 27.45456314086914
O : 92177.00645574056
92226.97741660144
AC
0.8962171157

kkset time 12.857307195663452
O : 1.9169358161737113
20.244709062778643
AC
0.9073412
TE
[ 18.35279914]
0.786402947363  wupsum
1.00000006036  wsum!
21  wupdic_len
O : 1.897643697002479
21.151400346890885
TE :  [ 19.28348786]
AS :  0.908011982186
O : 1.895885200398258
22.49521590266869
TE :  [ 20.6281839]
AS :  0.909392188248
O : 40745.2934651589
40769.82650124308
TE :  [ 24.56682881]
AS :  0.914759958011
O : 54803.66295373555
54829.01057446084
TE :  [ 25.38094235]
AS :  0.912648560159
Bs infeasible
clexsolve time :  2.1618099212646484
TE infeasible
kkset time 17.27150797843933
O : 1.9142789390224468
20.21085409330989
AC
0.907922683
TE
[ 18.32319813]
0.760334294595  wupsum
1.00000004683  wsum!
20  wupdic_len
O : 1.8865780393550207
21.389015128097483
TE :  [ 19.53602293]
AS :  0.908435183743
O : 1.8880994432328957
23.252578067636502
TE :  [ 21.40000684]
AS :  0.911225193222
O : 12579.789905957492
12604.686614883482
TE :  [ 24.93696307]
AS :  0.916239863336
O : 84452.08438980866
84477.4113

kkset time 12.64277195930481
O : 1.7387866790654534
24.229703253740094
AC
0.927405579717
TE
[ 22.52417487]
0.719259829357  wupsum
1.00000004304  wsum!
17  wupdic_len
O : 1.7585660360923654
27.15279746383825
TE :  [ 25.43426774]
AS :  0.927732875436
O : 1.7388025225341786
24.878619916985656
TE :  [ 23.1782734]
AS :  0.927218984
O : 1.744776609292948
25.739949886028942
TE :  [ 24.03416107]
AS :  0.927521190323
O : 1.7510480858212265
26.335514988430493
TE :  [ 24.62335402]
AS :  0.927521379382
O : 1.7552900026015472
26.700465294809298
TE :  [ 24.98424189]
AS :  0.927731172384
O : 1.757008561142043
26.91839849043465
TE :  [ 25.20091986]
AS :  0.927732048104
BS Feasible!!!
clexsolve time :  3.454329013824463
obj_val
25.2050369315
cplex_val
26.91839849043465
kkset time 19.21437406539917
O : 1.740585951324419
27.667172331525546
AC
0.928465718005
TE
[ 25.96607976]
clexsolve time :  0.4509890079498291
obj_val
25.9520480313
cplex_val
27.667172331525546
kkset time 23.661570072174072
cplex no solu

kkset time 13.00095796585083
O : 1.9745157518893208
22.897090332058745
AC
0.878928033
TE
[ 20.91016421]
0.747913768518  wupsum
1.00000008021  wsum!
19  wupdic_len
O : 1.9778192156003507
25.08811823576449
TE :  [ 23.09504]
AS :  0.878928033
O : 1.9890444951698356
27.990340621585755
TE :  [ 25.97928852]
AS :  0.882016713885
O : 1.9814320158402474
26.34532128060564
TE :  [ 24.34535343]
AS :  0.879393853332
O : 1.9840812878167344
27.110059754425265
TE :  [ 25.10567018]
AS :  0.880623706115
O : 1.982690815205943
26.715098454327563
TE :  [ 24.71297088]
AS :  0.880104086214
O : 1.9833834894573024
26.90940279583539
TE :  [ 24.90614691]
AS :  0.88027580945
BS Feasible!!!
clexsolve time :  2.310889959335327
obj_val
25.0902129728
cplex_val
27.110059754425265
kkset time 16.556945085525513
O : 1.975650385542374
25.312936947297015
AC
0.89453312
TE
[ 23.32792887]
0.767446522899  wupsum
1.00000006632  wsum!
19  wupdic_len
O : 1.9777319154659196
27.292507486449423
TE :  [ 25.30145884]
AS :  0.894840442

kkset time 11.713319778442383
O : 1.8660282958402399
21.071954277314827
AC
0.869706125518
TE
[ 19.18449096]
0.809760279513  wupsum
1.00000013836  wsum!
21  wupdic_len
O : 1.8612157245870846
22.473473598816202
TE :  [ 20.59227648]
AS :  0.873907022484
O : 1.8595397496424426
24.28337964175517
TE :  [ 22.40393258]
AS :  0.878040243734
O : 4486.00265874643
4513.328201763474
TE :  [ 27.31286542]
AS :  0.879512585905
Bs infeasible
clexsolve time :  1.537369966506958
TE infeasible
kkset time 17.02163791656494
O : 1.8687050496778528
20.48950544816749
AC
0.868123561095
TE
[ 18.60007654]
0.779513841165  wupsum
1.00000013924  wsum!
21  wupdic_len
O : 1.8608231899148298
22.245593955741406
TE :  [ 20.36616396]
AS :  0.871636434599
O : 1.858865929981286
24.396850279890334
TE :  [ 22.521184]
AS :  0.876458396029
O : 3514.2356159999417
3541.330277002494
TE :  [ 27.0850546]
AS :  0.878996142624
Bs infeasible
clexsolve time :  1.4902050495147705
TE infeasible
kkset time 21.77517795562744
O : 1.869355479

kkset time 12.983595132827759
cplex no solution
kkset time 17.762206077575684
O : 14416.877976486325
14455.431396457556
AC
0.916664312
TE
[ 38.55541535]
infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  0.5089352130889893
cplex no solution
kkset time 23.566160917282104
O : 14428.737308126241
14465.96361854163
AC
0.916307953
TE
[ 37.22720287]
infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  0.3837318420410156
cplex no solution
kkset time 27.351298093795776
O : 14181.60660141673
14217.160297446357
AC
0.915426886
TE
[ 35.55436638]
infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  0.4289247989654541
cplex no solution
kkset time 29.474425077438354
O : 14717.473233795024
14752.248137163737
AC
0.915615047
TE
[ 34.77354488]
infeasible
st_6 infeasible and infeasible sector is :  Financials
clexsolve time :  0.4354526996612549
cplex no solution
kkset time 33.96038818359375
O : 16418.69603902105

In [10]:
len(sol_all)

10

In [43]:
times={}
times['pre']=pre
times['step1']=step1
times['step2']=step2
times['step2+3']=step3
times['step3']=np.array(times['step2+3'])-np.array(times['step2'])

times=pd.DataFrame(times)
times2=times[times['step2']>0]
times2.describe()

,pre,step1,step2,step2+3,step3
count,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.229979,12.405187,1.330605,21.481214,20.150609
std,0.006353,0.078485,1.091025,1.111218,0.661615
min,0.224360,12.324312,0.304327,19.951683,18.885577
25%,0.226253,12.352077,0.340112,20.481807,19.712844
50%,0.227516,12.383123,0.390811,21.205083,20.056551
75%,0.232465,12.417278,2.362381,22.291900,20.557803
max,0.254310,12.672548,2.920118,23.974590,21.645249


In [11]:
obj_Update = [33.891587285731134, 25.09299329561036, 24.984764546200704, 24.966975825718855, 25.24671235679147, 25.158038999590666, 25.14073766356739, 25.0744257270579, 25.103619932228014]

In [12]:
TE_list=[22.87835501,70.22758985,27.29357997,23.41346661,24.14684022,25.18932616,24.53572075]
AS_list=[0.893388450961,0.893174573,0.893877522666,0.893388448546,0.893880492111,0.893879951624,0.893880528913]
obj_bi = [22.82923962950044,None,27.246690713625878,23.363384835018515,24.096623164811717,25.14073766356739,24.486286272271638]

In [18]:
len(TE_list)

7

In [13]:
x=range(10)

In [21]:
for i,j in zip(x,obj_Update):
    print((i,j))

(0, 33.891587285731134)
(1, 25.09299329561036)
(2, 24.984764546200704)
(3, 24.966975825718855)
(4, 25.24671235679147)
(5, 25.158038999590666)
(6, 25.14073766356739)
(7, 25.0744257270579)
(8, 25.103619932228014)


In [ ]:
r=[]
for i in range(len(list_feasi_obj)):
    if len(list_feasi_obj[i])>0:
        r.append(min(list_feasi_obj[i]))
        
len(r)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

#업데이트 다돌린 best값들만 비교
# r = [46.649545356782696, 29.419520394195146, 26.61847816786552, 27.826323522391554, 25.110385185009903, 25.81627500670225, 32.3072150037932, 28.52931408961155, 25.547458649598738, 46.649545356782696, 29.419520394195146, 26.61847816786552, 27.826323522391554]

#첫번째 나온 값들만 비교
#r = [46.649545356782696, 29.419520394195146, 26.61847816786552, 27.826323522391554, 25.110385185009903, 25.81627500670225, 32.3072150037932, 28.52931408961155, 25.547458649598738, 46.649545356782696, 29.419520394195146, 26.61847816786552, 27.826323522391554, 62.477376187441145, 52.51997145790738,  26.913498875674144, 27.997222600023196, 32.3167428569686, 37.25637523730125, 35.01942161049703]
# g = [31.56857694389234, 32.82558073084614, 31.474873540752014, 28.07668599755237, 27.296381684143928, 32.98458784268205, 31.85044044324648, 31.893165815380456, 25.854286554220405, 25.76949930030946, 32.98458784268205, 31.85044044324648, 31.893165815380456, 31.768048400971097, 31.82253080188232, 25.371056894268918, 27.45591403727919, 28.851552883205944, 27.492230645272517, 28.802294967910402]
c = []
all_data = [r]+[g]+[c]

# plot violin plot
axes.boxplot(all_data, showfliers =False)
axes.set_title('objective value distribution')
plt.setp(axes, xticks=[y+1 for y in range(len(all_data))], xticklabels=['RANDOM', 'GAN', 'CPLEX'])
plt.show()